# Imports

In [193]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 255)
from bs4 import BeautifulSoup
import nltk
from nltk.stem.porter import *
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ThibJacq\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [194]:
nltk.download('stopwords', quiet=True)

True

In [195]:
df_raw = pd.read_csv('./data/esa_csvs_raw.txt', sep='\t')

In [196]:
df_raw.iloc[0]

csvFileName                                                                                                                                                                                                                                        1059614_14_lattice-v_1.csv
csvText        [["Survey Date", "Aboriginal Communities", "Detail"], ["July 9 to 12, 2011", "Blueberry River First Nation <s>•</s>Halfway River First Nation <s>•</s>McLeod Lake Indian Band <s>•</s>North East Métis Association <s>•</s>", "Summer 2011 open water aquat...
Name: 0, dtype: object

In [197]:
df_raw['csvText'] = df_raw['csvText'].str.slice(0, 30_000) # limit to 30k chars since Excel can't handle more than 32k

In [198]:
assert (df_raw['csvText'].str.len()).max() <= 30_000 # sanity check

In [199]:
df_index = pd.read_csv('./data/ESA_website_ENG_2021_03_04_final_utf_8.csv', index_col=0, encoding='utf-8')

In [200]:
df_index.rename(columns={"Old CSV Filename": "csvFileName"}, inplace=True)

In [201]:
df_index.iloc[133:135] # sanity check

,Index,Title,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,Data ID,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,CSV Download URL,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,bad_csv,CSV Filename,csvFileName
133,9267,Table 2-1 Visual Sensitivity Ratings along Project RoW,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-25 ESA_Appendix_L_Part3of3 (A3Q6I1),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6I1,1059815,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059815,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix L: Socio-economic Technical Data Report,24.0,"Human, Technology, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-2-1-visual-sensitivity-ratings-along-project_pt-1_pg-8_doc-num-A3Q6I1.csv,8,22.0,2.77,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-2-1-visual-sensitivity-ratings-along-project_pt-1_pg-8_doc-num-A3Q6I1.csv,1059815_8_lattice-v_1.csv
134,9268,Table 6-1 Valued Components and Key Indicators – Soils,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-2 Volume 4 - ESA_Main_Part_1_of_5 (A3Q6F8),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6F8,1059854,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059854,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,"Table of Contents, Abbreviations, Section 1: Introduction, Section 2: Project Description, Section 3: Consultation and Engagement, Section 4: Route and Facility Site Selection, Section 5: Assessment Methods and Environmental Setting, Section 6: Soils,...",1.0,"Land, Water, Vegetation, Traditional Knowledge, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-6-1-valued-components-and-key-indicators--so_pt-1_pg-107_doc-num-A3Q6F8.csv,107,199.0,5.11,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-6-1-valued-components-and-key-indicators--so_pt-1_pg-107_doc-num-A3Q6F8.csv,1059854_107_lattice-v_1.csv


In [202]:
df_merged = df_raw[df_raw['csvFileName'].isin(list(df_index['csvFileName']))].reset_index(drop=True)

In [203]:
df_joined = pd.merge(df_index, df_merged, how='left', on='csvFileName')

In [204]:
df_joined.head()

,Index,Title,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,Data ID,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,CSV Download URL,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,bad_csv,CSV Filename,csvFileName,csvText
0,9134,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABORIGINAL FIELD STUDY PARTICIPATION FOR THE PROJECT,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-3-summary-of-aquatics-field-work-and-aborigi_pt-1_pg-14_doc-num-A3Q6H2.csv,14,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-3-summary-of-aquatics-field-work-and-aborigi_pt-1_pg-14_doc-num-A3Q6H2.csv,1059614_14_lattice-v_1.csv,"[[""Survey Date"", ""Aboriginal Communities"", ""Detail""], [""July 9 to 12, 2011"", ""Blueberry River First Nation <s>•</s>Halfway River First Nation <s>•</s>McLeod Lake Indian Band <s>•</s>North East Métis Association <s>•</s>"", ""Summer 2011 open water aquat..."
1,9135,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG THE NORTH MONTNEY MAINLINE (AITKEN CREEK SECTION),Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-4-summary-of-watercourse-crossings-along-the_pt-1_pg-17_doc-num-A3Q6H2.csv,17,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossings-along-the_pt-1_pg-17_doc-num-A3Q6H2.csv,1059614_17_lattice-v_1.csv,"[[""Field Site No."", ""Name, Legal Location"", ""UTM Coordinates (NAD83, Zone 10)"", ""Latitude/Longitude (DD-MM-SS), Elevation (m asl)"", ""Fish Stream Classification <s>1,7 </s>"", ""Fish and Wildlife Timing Window – Low Risk <s>2,7 </s>"", ""Riparian Managem..."
2,9136,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG THE NORTH MONTNEY MAINLINE (AITKEN CREEK SECTION),Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-4-summary-of-watercourse-crossings-along-the_pt-2_pg-18_doc-num-A3Q6H2.csv,18,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-

In [205]:
len(df_joined) # 28891

28891

In [206]:
del df_raw
del df_index
del df_merged

## Adding New Projects to the Index File

In [207]:
df_joined.rename(columns={"csvText": "text"}, inplace=True)
df_joined['label'] = -1

In [208]:
df_joined["text"] = df_joined["Title"] + " " + df_joined["text"]

In [209]:
df_joined['text'] = df_joined['text'].apply(lambda x: BeautifulSoup(x, "html.parser").get_text(separator=' ')) # remove html tags
df_joined['text'] = df_joined['text'].replace('[^a-zA-Z0-9 ]', ' ', regex=True) # remove all non-alpha-numeric characters
df_joined['text'] = df_joined['text'].replace('\w{25,}', ' ', regex=True)
df_joined['text'] = df_joined['text'].replace('cid\d+', ' ', regex=True)
df_joined['text'] = df_joined['text'].replace(' s ', ' ', regex=True)
df_joined['text'] = df_joined['text'].replace(' +', ' ', regex=True) # remove all extra spaces in text

In [210]:
df_joined.text[103]

'Table 4 28 Assessed Value of an Average Single Family Home 2011 to 2013 Community 2011 2012 2013 2011 to 2013 increase Dawson Creek 212 870 229 000 238 000 12 Fort St John 274 035 305 000 326 000 19 Chetwynd 156 170 218 000 229 000 47 Hudson Hope 132 270 161 000 175 000 32 Taylor 208 098 247 000 281 000 35 SOURCE City of Dawson Creek 2013 '

In [213]:
df_joined.to_csv('./data/esa_index_with_table_text_no_labels.csv', index=False, encoding='utf-8')

In [212]:
from sklearn.model_selection import train_test_split
train_df, df_for_labeling = train_test_split(df_joined, test_size=1000, random_state=42, shuffle=True)

In [269]:
print(len(train_df))
print(len(df_for_labeling))

27891
1000


In [270]:
train_df.head(2)

,Index,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,...,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,Missing CSV,CSV Filename,csvFileName,text,label
25896,35030,Table,Application for the Enbridge Northern Gateway ...,Northern Gateway,5/27/2010,"Northern Gateway Pipelines Inc., as General Pa...",Oil,B3-1 - Vol 6A P1 - Pipelines and Tank Terminal...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A1T0F1,...,184.0,4.85,Yes,nrthrngtwy,http://www.cer-rec.gc.ca/esa-ees/nrthrngtwy.zip,False,nrthrngtwy_table-2-6-estimated-barge-traffic-f...,620130_44_lattice-v_1.csv,Table 26 Estimated Barge Traffic for Marine Te...,-1
8409,17543,Table,Application for the 2017 NGTL System Expansion,2017 NGTL System Expansion,3/31/2015,NOVA Gas Transmission Ltd.,Gas,B3-33_ESA_Appendix_4_AQ_TDR_Part1of2 - A4K2T9,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A4K2T9,...,39.0,5.67,Yes,2017ngtl,http://www.cer-rec.gc.ca/esa-ees/2017ngtl.zip,False,2017ngtl_table-14-alces-river-unit-addition-ai...,2758416_26_lattice-v_1.csv,TABLE 14 ALCES RIVER UNIT ADDITION AIR DISPERS...,-1


In [271]:
df_for_labeling.head(2)

,Index,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,...,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,Missing CSV,CSV Filename,csvFileName,text,label
8889,18023,Table,Application for the 2017 NGTL System Expansion,2017 NGTL System Expansion,3/31/2015,NOVA Gas Transmission Ltd.,Gas,B2-15_ESA_Main_Sec_8_Part1of2 - A4K2S0,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A4K2S0,...,472.0,6.50,Yes,2017ngtl,http://www.cer-rec.gc.ca/esa-ees/2017ngtl.zip,False,2017ngtl_table-8a-2-reasonably-forseeable-deve...,2758825_359_lattice-v_1.csv,TABLE 8A2 REASONABLY FORSEEABLE DEVELOPMENTS I...,-1
27753,36887,Table,Application for the Northwest Mainline Expansion,Northwest Mainline,4/29/2011,NOVA Gas Transmission Ltd.,Gas,B3-38 - 7212_App8_EPP - A1Y9J1,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A1Y9J1,...,194.0,4.43,Yes,nrthwstmnln,http://www.cer-rec.gc.ca/esa-ees/nrthwstmnln.zip,False,nrthwstmnln_table-1-resource-specific-mitigati...,684515_22_lattice-v_1.csv,TABLE 1 RESOURCESPECIFIC MITIGATION TABLE Loca...,-1


In [272]:
df_for_labeling.to_csv('./mydata/esa_index_for_labeling.csv')

In [274]:
df_for_labeling = pd.read_csv('./mydata/esa_index_for_labeling.csv', index_col=0)
df_train = pd.read_csv('./data/esa_index_with_table_text_no_labels.csv', index_col=0)

In [275]:
df_train.head()

,Index,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,...,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,Missing CSV,CSV Filename,csvFileName,text,label
0,9134,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,...,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-3-summary-of-aquatics-field-wor...,1059614_14_lattice-v_1.csv,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABO...,-1
1,9135,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,...,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_17_lattice-v_1.csv,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,-1
2,9136,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,...,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_18_lattice-v_1.csv,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,-1
3,9137,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,...,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_19_lattice-v_1.csv,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,-1
4,9138,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,...,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_20_lattice-v_1.csv,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,-1


In [276]:
# here we will truncate the number of tokens in the table text so that 
# CUDA is not out of memory when we use zero-shot to label our data
# this may or may not be necessary, should revisit later
max_token_num = 128

In [277]:
df_for_labeling['full_text'] = df_for_labeling['text']
for row in df_for_labeling.itertuples(index=True):
    tokenized_table_text = word_tokenize(df_for_labeling['text'][row.Index])
    if len(tokenized_table_text) > max_token_num:
        df_for_labeling['text'][row.Index] = TreebankWordDetokenizer().detokenize(tokenized_table_text[0:max_token_num])
        
df_for_labeling.head(2)

<ipython-input-277-222a4dffab34>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_for_labeling['text'][row.Index] = TreebankWordDetokenizer().detokenize(tokenized_table_text[0:max_token_num])


,Index,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,...,PDF Size,PDF Outline,Download folder name,Zipped Project Link,Missing CSV,CSV Filename,csvFileName,text,label,full_text
8889,18023,Table,Application for the 2017 NGTL System Expansion,2017 NGTL System Expansion,3/31/2015,NOVA Gas Transmission Ltd.,Gas,B2-15_ESA_Main_Sec_8_Part1of2 - A4K2S0,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A4K2S0,...,6.50,Yes,2017ngtl,http://www.cer-rec.gc.ca/esa-ees/2017ngtl.zip,False,2017ngtl_table-8a-2-reasonably-forseeable-deve...,2758825_359_lattice-v_1.csv,TABLE 8A2 REASONABLY FORSEEABLE DEVELOPMENTS I...,-1,TABLE 8A2 REASONABLY FORSEEABLE DEVELOPMENTS I...
27753,36887,Table,Application for the Northwest Mainline Expansion,Northwest Mainline,4/29/2011,NOVA Gas Transmission Ltd.,Gas,B3-38 - 7212_App8_EPP - A1Y9J1,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A1Y9J1,...,4.43,Yes,nrthwstmnln,http://www.cer-rec.gc.ca/esa-ees/nrthwstmnln.zip,False,nrthwstmnln_table-1-resource-specific-mitigati...,684515_22_lattice-v_1.csv,TABLE 1 RESOURCESPECIFIC MITIGATION TABLE Loca...,-1,TABLE 1 RESOURCESPECIFIC MITIGATION TABLE Loca...


In [278]:
df_train['full_text'] = df_train['text']
for row in df_train.itertuples(index=True):
    tokenized_table_text = word_tokenize(df_train['text'][row.Index])
    if len(tokenized_table_text) > max_token_num:
        df_train['text'][row.Index] = TreebankWordDetokenizer().detokenize(tokenized_table_text[0:max_token_num])
        
df_train.head(2)

<ipython-input-278-e08837a51249>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['text'][row.Index] = TreebankWordDetokenizer().detokenize(tokenized_table_text[0:max_token_num])


,Index,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,...,PDF Size,PDF Outline,Download folder name,Zipped Project Link,Missing CSV,CSV Filename,csvFileName,text,label,full_text
0,9134,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,...,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-3-summary-of-aquatics-field-wor...,1059614_14_lattice-v_1.csv,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABO...,-1,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABO...
1,9135,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,...,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_17_lattice-v_1.csv,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,-1,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...


In [279]:
for row in df_for_labeling.itertuples():
    tokenized_table_text = word_tokenize(df_for_labeling['text'][row.Index])
    assert len(tokenized_table_text) <= max_token_num, f"You have at least one table with more than {max_token_num} tokens."

In [280]:
df_for_labeling.to_csv(f'./mydata/zero_shot_esa_index_for_labeling_{max_token_num}_max_tokens.csv')

In [281]:
df_joined.to_csv(f'./data/zero_shot_esa_index_train_{max_token_num}_max_tokens.csv')